# Using AWS Bedrock for RAG

In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
import os

import boto3
from zenml.service_connectors.service_connector import ServiceConnector


def get_boto_client() -> ServiceConnector:
    # zc = Client()
    # return zc.get_service_connector_client(
    #     name_id_or_prefix="0b04bcae-efc9-4044-a1c2-b86281cb0820",  # TODO: pull this out into config file
    #     resource_type="aws-generic",
    # ).connect()
    os.environ["AWS_PROFILE"] = "zenml-devel"
    return boto3.Session()

## Creating policies for Bedrock and OpenSearch Serverless

In [4]:
import json

# suffix = random.randrange(200, 900)
boto3_session = get_boto_client()
region_name = "us-east-1"

In [ ]:
iam_client = boto3_session.client("iam", region_name=region_name)
account_number = (
    boto3_session.client("sts", region_name=region_name)
    .get_caller_identity()
    .get("Account")
)
identity = boto3_session.client(
    "sts", region_name=region_name
).get_caller_identity()["Arn"]

In [ ]:
encryption_policy_name = f"bedrock-sample-rag-sp-{suffix}"
network_policy_name = f"bedrock-sample-rag-np-{suffix}"
access_policy_name = f"bedrock-sample-rag-ap-{suffix}"
bedrock_execution_role_name = (
    f"AmazonBedrockExecutionRoleForKnowledgeBase_{suffix}"
)
fm_policy_name = f"AmazonBedrockFoundationModelPolicyForKnowledgeBase_{suffix}"
s3_policy_name = f"AmazonBedrockS3PolicyForKnowledgeBase_{suffix}"
sm_policy_name = f"AmazonBedrockSecretPolicyForKnowledgeBase_{suffix}"
oss_policy_name = f"AmazonBedrockOSSPolicyForKnowledgeBase_{suffix}"


def create_oss_policy_attach_bedrock_execution_role(
    collection_id, bedrock_kb_execution_role
):
    # define oss policy document
    oss_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": ["aoss:APIAccessAll"],
                "Resource": [
                    f"arn:aws:aoss:{region_name}:{account_number}:collection/{collection_id}"
                ],
            }
        ],
    }
    oss_policy = iam_client.create_policy(
        PolicyName=oss_policy_name,
        PolicyDocument=json.dumps(oss_policy_document),
        Description="Policy for accessing opensearch serverless",
    )
    oss_policy_arn = oss_policy["Policy"]["Arn"]
    print("Opensearch serverless arn: ", oss_policy_arn)

    iam_client.attach_role_policy(
        RoleName=bedrock_kb_execution_role["Role"]["RoleName"],
        PolicyArn=oss_policy_arn,
    )
    return None


def create_policies_in_oss(
    vector_store_name, aoss_client, bedrock_kb_execution_role_arn
):
    encryption_policy = aoss_client.create_security_policy(
        name=encryption_policy_name,
        policy=json.dumps(
            {
                "Rules": [
                    {
                        "Resource": ["collection/" + vector_store_name],
                        "ResourceType": "collection",
                    }
                ],
                "AWSOwnedKey": True,
            }
        ),
        type="encryption",
    )

    network_policy = aoss_client.create_security_policy(
        name=network_policy_name,
        policy=json.dumps(
            [
                {
                    "Rules": [
                        {
                            "Resource": ["collection/" + vector_store_name],
                            "ResourceType": "collection",
                        }
                    ],
                    "AllowFromPublic": True,
                }
            ]
        ),
        type="network",
    )
    access_policy = aoss_client.create_access_policy(
        name=access_policy_name,
        policy=json.dumps(
            [
                {
                    "Rules": [
                        {
                            "Resource": ["collection/" + vector_store_name],
                            "Permission": [
                                "aoss:CreateCollectionItems",
                                "aoss:DeleteCollectionItems",
                                "aoss:UpdateCollectionItems",
                                "aoss:DescribeCollectionItems",
                            ],
                            "ResourceType": "collection",
                        },
                        {
                            "Resource": ["index/" + vector_store_name + "/*"],
                            "Permission": [
                                "aoss:CreateIndex",
                                "aoss:DeleteIndex",
                                "aoss:UpdateIndex",
                                "aoss:DescribeIndex",
                                "aoss:ReadDocument",
                                "aoss:WriteDocument",
                            ],
                            "ResourceType": "index",
                        },
                    ],
                    "Principal": [identity, bedrock_kb_execution_role_arn],
                    "Description": "Easy data policy",
                }
            ]
        ),
        type="data",
    )
    return encryption_policy, network_policy, access_policy

In [ ]:
import json

sts_client = boto3_session.client("sts", region_name=region_name)
region_name = "us-east-1"

In [ ]:
region_name

In [ ]:
bedrock_agent_client = boto3_session.client(
    "bedrock-agent", region_name=region_name
)
bedrock_agent_runtime_client = boto3_session.client(
    "bedrock-agent-runtime", region_name=region_name
)

service = "aoss"
s3_client = boto3_session.client("s3")
account_id = sts_client.get_caller_identity()["Account"]
s3_suffix = f"{region_name}-{account_id}"

In [ ]:
bucket_name = "bedrock-zenml-rag-docs"

vector_store_name = f"bedrock-vectordb-rag-{suffix}"
index_name = f"bedrock-vectordb-rag-index-{suffix}"
aoss_client = boto3_session.client(
    "opensearchserverless", region_name=region_name
)

In [ ]:
def create_bedrock_execution_role(bucket_name: str):
    foundation_model_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "bedrock:InvokeModel",
                ],
                "Resource": [
                    f"arn:aws:bedrock:{region_name}::foundation-model/amazon.titan-embed-text-v1",
                    f"arn:aws:bedrock:{region_name}::foundation-model/amazon.titan-embed-text-v2:0",
                ],
            }
        ],
    }

    s3_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": ["s3:GetObject", "s3:ListBucket"],
                "Resource": [
                    f"arn:aws:s3:::{bucket_name}",
                    f"arn:aws:s3:::{bucket_name}/*",
                ],
                "Condition": {
                    "StringEquals": {
                        "aws:ResourceAccount": f"{account_number}"
                    }
                },
            }
        ],
    }

    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {"Service": "bedrock.amazonaws.com"},
                "Action": "sts:AssumeRole",
            }
        ],
    }

    try:
        # Try to get the role if it already exists
        existing_role = iam_client.get_role(
            RoleName=bedrock_execution_role_name
        )
        print(f"Role {bedrock_execution_role_name} already exists.")
        bedrock_kb_execution_role = existing_role
    except iam_client.exceptions.NoSuchEntityException:
        # If the role doesn't exist, create it
        print(f"Creating new role: {bedrock_execution_role_name}")
        bedrock_kb_execution_role = iam_client.create_role(
            RoleName=bedrock_execution_role_name,
            AssumeRolePolicyDocument=json.dumps(assume_role_policy_document),
            Description="Amazon Bedrock Knowledge Base Execution Role for accessing OSS and S3",
            MaxSessionDuration=3600,
        )

    # Use put_role_policy instead of create_policy and attach_role_policy
    iam_client.put_role_policy(
        RoleName=bedrock_kb_execution_role["Role"]["RoleName"],
        PolicyName="foundation_model_policy",
        PolicyDocument=json.dumps(foundation_model_policy_document),
    )

    iam_client.put_role_policy(
        RoleName=bedrock_kb_execution_role["Role"]["RoleName"],
        PolicyName="s3_policy",
        PolicyDocument=json.dumps(s3_policy_document),
    )

    return bedrock_kb_execution_role

In [ ]:
bedrock_kb_execution_role = create_bedrock_execution_role(bucket_name)
bedrock_kb_execution_role_arn = bedrock_kb_execution_role["Role"]["Arn"]

## Create a collection & vector store using OpenSearch Serverless

In [ ]:
# create security, network and data access policies within OSS
encryption_policy, network_policy, access_policy = create_policies_in_oss(
    vector_store_name=vector_store_name,
    aoss_client=aoss_client,
    bedrock_kb_execution_role_arn=bedrock_kb_execution_role_arn,
)

In [ ]:
boto3_session = get_boto_client()

aoss_client = boto3_session.client(
    "opensearchserverless", region_name="us-east-1"
)

collection = aoss_client.create_collection(
    name=vector_store_name, type="VECTORSEARCH"
)

In [ ]:
aoss_client.list_collections(maxResults=15)

In [ ]:
import time

from rich import print as pp

pp(collection)
time.sleep(10)

In [ ]:
collection_id = collection["createCollectionDetail"]["id"]
host = f"{collection_id}.{region_name}.aoss.amazonaws.com"
pp(host)

In [ ]:
create_oss_policy_attach_bedrock_execution_role(
    collection_id=collection_id,
    bedrock_kb_execution_role=bedrock_kb_execution_role,
)

## Create vector index

In [ ]:
from opensearchpy import AWSV4SignerAuth, OpenSearch, RequestsHttpConnection

credentials = get_boto_client().get_credentials()
awsauth = auth = AWSV4SignerAuth(credentials, region_name, service)

index_name = f"bedrock-sample-index-{suffix}"
body_json = {
    "settings": {"index.knn": "true"},
    "mappings": {
        "properties": {
            "vector": {
                "type": "knn_vector",
                "dimension": 1536,
                "method": {
                    "name": "hnsw",
                    "engine": "faiss",
                    "space_type": "l2",
                    "parameters": {"ef_construction": 200, "m": 16},
                },
            },
            "text": {"type": "text"},
            "text-metadata": {"type": "text"},
        }
    },
}

# Build the OpenSearch client
oss_client = OpenSearch(
    hosts=[{"host": host, "port": 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300,
)
# # It can take up to a minute for data access rules to be enforced
time.sleep(60)

In [ ]:
# Create index
response = oss_client.indices.create(
    index=index_name, body=json.dumps(body_json)
)

print("\nCreating index:")
pp(response)

## Create Knowledge Base

In [ ]:
boto3_session = get_boto_client()
aoss_client = boto3_session.client(
    "opensearchserverless", region_name="us-east-1"
)


collections = aoss_client.list_collections(maxResults=15)
collection = collections["collectionSummaries"][0]

In [ ]:
collection_id = collection["id"]

opensearchServerlessConfiguration = {
    "collectionArn": collection["arn"],
    "vectorIndexName": index_name,
    "fieldMapping": {
        "vectorField": "vector",
        "textField": "text",
        "metadataField": "text-metadata",
    },
}

# # Different Chunking Strategies
# # FIXED_SIZE Chunking
chunkingStrategyConfiguration = {
    "chunkingStrategy": "FIXED_SIZE",
    "fixedSizeChunkingConfiguration": {
        "maxTokens": 512,
        "overlapPercentage": 20,
    },
}

# # HIERARCHICAL Chunking
# chunkingStrategyConfiguration = {
#     "chunkingStrategy": "HIERARCHICAL",
#     "hierarchicalChunkingConfiguration": {
#         "levelConfigurations": [{"maxTokens": 1500}, {"maxTokens": 300}],
#         "overlapTokens": 60,
#     },
# }

# # SEMANTIC Chunking
# chunkingStrategyConfiguration = {
#     "semanticChunkingConfiguration": {
#         "breakpointPercentileThreshold": 95,
#         "bufferSize": 1,
#         "maxTokens": 300,
#     }
# }


# S3
s3Configuration = {
    "bucketArn": f"arn:aws:s3:::{bucket_name}",
}

In [6]:
suffix = 556

In [7]:
boto3_session = get_boto_client()

bedrock_agent_client = boto3_session.client(
    "bedrock-agent", region_name=region_name
)

embeddingModelArn = f"arn:aws:bedrock:{region_name}::foundation-model/amazon.titan-embed-text-v1"

name = f"bedrock-sample-knowledge-base-{suffix}"
description = "Bedrock Knowledge Bases for Web URL and S3 Connector"
roleArn = f"AmazonBedrockExecutionRoleForKnowledgeBase_{suffix}"

Found credentials in shared credentials file: ~/.aws/credentials


In [8]:
# Create a KnowledgeBase
from retrying import retry


@retry(wait_random_min=1000, wait_random_max=2000, stop_max_attempt_number=2)
def create_knowledge_base_func():
    create_kb_response = bedrock_agent_client.create_knowledge_base(
        name=name,
        description=description,
        roleArn=roleArn,
        knowledgeBaseConfiguration={
            "type": "VECTOR",
            "vectorKnowledgeBaseConfiguration": {
                "embeddingModelArn": embeddingModelArn
            },
        },
        storageConfiguration={
            "type": "OPENSEARCH_SERVERLESS",
            "opensearchServerlessConfiguration": opensearchServerlessConfiguration,
        },
    )
    return create_kb_response["knowledgeBase"]

In [9]:
try:
    kb = create_knowledge_base_func()
except Exception as err:
    print(f"{err=}, {type(err)=}")

err=NameError("name 'opensearchServerlessConfiguration' is not defined"), type(err)=<class 'NameError'>


In [ ]:
# import json
# from botocore.exceptions import ClientError

# policy_name = "BedrockKnowledgeBaseAccess"
# policy_document = {
#     "Version": "2012-10-17",
#     "Statement": [
#         {
#             "Effect": "Allow",
#             "Action": [
#                 "bedrock:CreateKnowledgeBase",
#                 "bedrock:DeleteKnowledgeBase",
#                 "bedrock:GetKnowledgeBase",
#                 "bedrock:ListKnowledgeBases",
#                 "bedrock:UpdateKnowledgeBase"
#             ],
#             "Resource": "*"
#         }
#     ]
# }

# try:
#     response = iam_client.create_policy(
#         PolicyName=policy_name,
#         PolicyDocument=json.dumps(policy_document)
#     )
#     policy_arn = response['Policy']['Arn']
#     print(f"Created policy: {policy_arn}")

#     iam_client.attach_role_policy(
#         RoleName="zenml-connectors",
#         PolicyArn=policy_arn
#     )
#     print(f"Attached policy to role: zenml-connectors")
# except ClientError as e:
#     print(f"An error occurred: {e}")

In [ ]:
pp(kb)

In [ ]:
# Get KnowledgeBase
get_kb_response = bedrock_agent_client.get_knowledge_base(
    knowledgeBaseId=kb["knowledgeBaseId"]
)

## Create Data Source for Knowledge Base

In [ ]:
# Create a S3 DataSource in KnowledgeBase
create_ds_response = bedrock_agent_client.create_data_source(
    name=name,
    description=description,
    knowledgeBaseId=kb["knowledgeBaseId"],
    dataDeletionPolicy="DELETE",
    dataSourceConfiguration={
        # # For S3
        "type": "S3",
        "s3Configuration": s3Configuration,
        # # For Web URL
        # "type": "WEB",
        # "webConfiguration":webConfiguration
    },
    vectorIngestionConfiguration={
        "chunkingConfiguration": chunkingStrategyConfiguration
    },
)

ds = create_ds_response["dataSource"]
pp(ds)

In [ ]:
# get s3 datasource
bedrock_agent_client.get_data_source(
    knowledgeBaseId=kb["knowledgeBaseId"], dataSourceId=ds["dataSourceId"]
)

## Start ingestion job

In [ ]:
time.sleep(10)

# Start an ingestion job
start_job_response = bedrock_agent_client.start_ingestion_job(
    knowledgeBaseId=kb["knowledgeBaseId"], dataSourceId=ds["dataSourceId"]
)

In [ ]:
job = start_job_response["ingestionJob"]
pp(job)

In [ ]:
job

In [ ]:
# Get job
while job["status"] != "COMPLETE":
    get_job_response = bedrock_agent_client.get_ingestion_job(
        knowledgeBaseId=kb["knowledgeBaseId"],
        dataSourceId=ds["dataSourceId"],
        ingestionJobId=job["ingestionJobId"],
    )
    job = get_job_response["ingestionJob"]

pp(job)

# time.sleep(40)

In [ ]:
kb_id = kb["knowledgeBaseId"]
print(kb_id)

## Test the Knowledge Base

In [ ]:
# try out KB using RetrieveAndGenerate API
# model_id = "anthropic.claude-3-sonnet-20240229-v1:0"  # <Change it to any model of your choice which is supported by KB>
model_id = "anthropic.claude-3-haiku-20240307-v1:0"
model_arn = f"arn:aws:bedrock:us-east-1::foundation-model/{model_id}"

### Use the RetrieveAndGenerate API

In [ ]:
query = "What orchestrators does ZenML support?"
response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={"text": query},
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            "knowledgeBaseId": kb_id,
            "modelArn": model_arn,
        },
    },
)

generated_text = response["output"]["text"]

print(generated_text)

### Print out the Source Attributions

In [ ]:
## print out the source attribution/citations from the original documents to see if the response generated belongs to the context.
citations = response["citations"]
contexts = []
for citation in citations:
    retrievedReferences = citation["retrievedReferences"]
    for reference in retrievedReferences:
        contexts.append(reference["content"]["text"])

print(contexts[0])

## Use the Retrieve API

In [ ]:
# retrieve api for fetching only the relevant context.
# query = "How many new positions were opened across Amazon's fulfillment and delivery network?"

relevant_documents = bedrock_agent_runtime_client.retrieve(
    retrievalQuery={"text": query},
    knowledgeBaseId=kb_id,
    retrievalConfiguration={
        "vectorSearchConfiguration": {
            "numberOfResults": 3  # will fetch top 3 documents which matches closely with the query.
        }
    },
)

In [ ]:
# print(relevant_documents["retrievalResults"])

for doc in relevant_documents["retrievalResults"]:
    print(doc["content"]["text"])

# Clean Up

In [ ]:
# # Delete KnowledgeBase
# bedrock_agent_client.delete_data_source(
#     dataSourceId=ds["dataSourceId"], knowledgeBaseId=kb["knowledgeBaseId"]
# )
# bedrock_agent_client.delete_knowledge_base(
#     knowledgeBaseId=kb["knowledgeBaseId"]
# )
# oss_client.indices.delete(index=index_name)
# aoss_client.delete_collection(id=collection_id)
# aoss_client.delete_access_policy(
#     type="data", name=access_policy["accessPolicyDetail"]["name"]
# )
# aoss_client.delete_security_policy(
#     type="network", name=network_policy["securityPolicyDetail"]["name"]
# )
# aoss_client.delete_security_policy(
#     type="encryption", name=encryption_policy["securityPolicyDetail"]["name"]
# )

In [ ]:
# def delete_iam_role_and_policies():
#     fm_policy_arn = f"arn:aws:iam::{account_number}:policy/{fm_policy_name}"
#     s3_policy_arn = f"arn:aws:iam::{account_number}:policy/{s3_policy_name}"
#     oss_policy_arn = f"arn:aws:iam::{account_number}:policy/{oss_policy_name}"
#     sm_policy_arn = f"arn:aws:iam::{account_number}:policy/{sm_policy_name}"

#     iam_client.detach_role_policy(
#         RoleName=bedrock_execution_role_name, PolicyArn=s3_policy_arn
#     )
#     iam_client.detach_role_policy(
#         RoleName=bedrock_execution_role_name, PolicyArn=fm_policy_arn
#     )
#     iam_client.detach_role_policy(
#         RoleName=bedrock_execution_role_name, PolicyArn=oss_policy_arn
#     )
#     iam_client.detach_role_policy(
#         RoleName=bedrock_execution_role_name, PolicyArn=sm_policy_arn
#     )
#     iam_client.delete_role(RoleName=bedrock_execution_role_name)
#     iam_client.delete_policy(PolicyArn=s3_policy_arn)
#     iam_client.delete_policy(PolicyArn=fm_policy_arn)
#     iam_client.delete_policy(PolicyArn=oss_policy_arn)
#     iam_client.delete_policy(PolicyArn=sm_policy_arn)
#     return 0


# delete_iam_role_and_policies()

Parts adapted and reworked from https://github.com/build-on-aws/llm-rag-vectordb-python/blob/main/bedrock-kb/KB_Bedrock_Accuracy_Improvement.ipynb